In [16]:
# import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from scipy import stats
import statsmodels.api as sm
import yfinance as yf
import dash
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
from datetime import date
import time
from dash.exceptions import PreventUpdate


ticker_list = pd.read_csv("tickers.csv", usecols = ["Symbol"])

def get_tickers():
    return([{'label': c, 'value': c} for c in ticker_list['Symbol'].drop_duplicates()])


# Dash app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)


app.layout = html.Div(
    [
        html.Center(html.H4('Dash App Demo')),
        html.Hr(),
        html.Div([
            html.Div([
                html.Label('Select Your Stock:'), 
                #html.Br(),
                dcc.Dropdown(
                id='ticker',
                placeholder='Enter a Ticker...',
                options = get_tickers(),
                value = ['AAPL'],
                multi = True,
                ),
            ], style = {'margin-right': '3em', 'width': '20em'}),
            html.Div([
                html.Label('Choose Your Time Period:'), 
                #html.Br(),
                dcc.DatePickerRange(
                id='date-range',
                start_date_placeholder_text='Select a date!',
                end_date_placeholder_text='Select a date!',
                start_date=date(2016,9,27),
                end_date=date(2021,9,24)
                ),
            ], style = {'margin-right': '3em'}),
            html.Div([
                html.Button('Submit', id='button'),
            ], style = {'margin-top': '25px'}),
        ], style = {'display': 'flex', 'flex-direction': 'row', 'padding': '1em'}),
        
        html.Center(id='ticker-plot'),
        html.Div(id='strategy-list', children=[
            html.Hr(),
            html.Label('Select Your Strategy:'), 
            html.Br(),
            dcc.Dropdown(
                id='strategy',
                options=[
                {'label': 'Filter Rules', 'value': 'FR'},
                {'label': 'Simple Moving Average', 'value': 'SMA'},
                {'label': 'Exponential Moving Average', 'value': 'EMA'},
                {'label': 'Channel Breakouts', 'value': 'CB'},
            ],
            #value='FR',
            searchable = True,
            ),
            html.Br(),
        ], style = {'visibility': 'hidden'}),
    
        html.Div(id='parameters', children=[
            html.Label('Choose your parameters:'), 
            html.Div(id='param-list', children=[
                html.Div(id='window-size', children=[
                    html.Label('Window Size (1-100):'),
                    dcc.Input(
                        id='WS',
                        type='number',
                        min=1,
                        max=100,
                        style={'width':'10em', 'margin-left':'1em'},
                        ),
                    ], style={'display':'none'}),
                html.Div(id='short-window-size', children=[
                    html.Label('Short Window Size (1-100):'),
                    dcc.Input(
                        id='SWS',
                        type='number',
                        min=1,
                        max=100,
                        style={'width':'10em', 'margin-left':'1em'},
                        ),
                    ], style={'display':'none'}),
                html.Div(id='long-window-size', children=[
                    html.Label('Long Window Size (1-100):'),
                    dcc.Input(
                        id='LWS',
                        type='number',
                        min=1,
                        max=100,
                        style={'width':'10em', 'margin-left':'1em'},
                        ),
                    ], style={'display':'none'}),
                html.Div(id='k-value', children=[
                    html.Label('k (0-5):'),
                    dcc.Input(
                        id='K',
                        type='number',
                        min=0,
                        max=5,
                        step=0.1,
                        style={'width':'10em', 'margin-left':'1em'},
                        ),
                    ], style={'display':'none'}),
                html.Div(id='L-value', children=[
                    html.Label('L (1-50):'),
                    dcc.Input(
                        id='L',
                        type='number',
                        min=1,
                        max=50,
                        style={'width':'10em', 'margin-left':'1em'},
                        ),
                    ], style={'display':'none'}),
                html.Div(id='threshold', children=[
                    html.Label('Threshold (0-1):'),
                    dcc.Input(
                        id='THRES',
                        type='number',
                        min=0,
                        max=1,
                        step=0.01,
                        style={'width':'10em', 'margin-left':'1em'},
                        ),
                    ], style={'display':'none'}),
            ], style={'display': 'flex', 'flex-direction': 'row', 'padding': '1em'}),
            
            html.Button('Test !', id='button2'),
        ], style={'visibility': 'hidden'}),
        
        html.Center(id='hist-result-plot'),
        html.Center(id='hist-result-table'),
        
        
        # store the intermediate data
        dcc.Store(id='prdata'),
    ],  
    style ={
        'margin': '2em',
        #'border-radius': '1em',
        #'border-style': 'solid', 
        'padding': '1em',
        #'background': '#ededed',
    }
)

@app.callback(
    Output('prdata', 'data'),
    Input('button', 'n_clicks'),
    State('ticker', 'value'),
    State('date-range', 'start_date'),
    State('date-range', 'end_date'), prevent_initial_call=True)
def download_data(n_clicks, ticker, start_d, end_d):
    if ticker and start_d and end_d:
        myTicker = ' '
        myTicker = myTicker.join(ticker)
        if len(ticker) == 1:
            hist = yf.download(myTicker, start=start_d, end=end_d)[['Adj Close']]
            hist = hist.rename(columns = {'Adj Close' : myTicker})
        else:
            hist = yf.download(myTicker, start=start_d, end=end_d)['Adj Close']
        if hist.empty:
            return None
        tic = hist.columns
        for i in range(len(tic)):
            hist['{}_r'.format(tic[i])] = np.log(hist[tic[i]]).diff()
        return hist.to_json(date_format='iso')
    else:
        return None


@app.callback(
    Output('ticker-plot', 'children'),
    Input('prdata', 'data'),
    State('ticker', 'value') , prevent_initial_call=True)
def update_graphs(data, ticker):
    if data:
        hist = pd.read_json(data)
        xformat = dict(rangeslider=dict(visible=True), type="date")

        # %% Price Graph generation
        fig1 = go.Figure()
        for i in range(len(ticker)):
            price = hist[[ticker[i]]].dropna()
            fig1.add_trace(go.Scatter(y=price[ticker[i]].to_list(), x=price.index.to_list(), name=ticker[i]))

        fig1.update_layout(
            title={
                'text': 'Historical Stock Closing Prices',
                'y': 0.95,
                'x': 0.5,
                'font': {'size': 22}
            },
            paper_bgcolor='white',
            plot_bgcolor='white',
            autosize=False,
            height = 450,
            xaxis={**xformat, **{
                'title': 'Closing Date',
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black',
            }},
            yaxis={
                'title': 'Price',
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black'
            },
            legend={
                'orientation': 'h',  # 图例横着排列还是竖着排列：h stands for horizontal
                # This orients the legend but it will result in an overlay of the graph
                # 'yanchor': 'top', 'y': 0.5,
                'yanchor': 'top', 'y': 1.15,  # 'top' 指上端在整个图里位置的比例（可以在0-1范围之外）
                'xanchor': 'right', 'x': 1,  # ‘right’指右端在整个图位置的比例
            },
            # this fixes the overlay
            margin={'t': 100},
        )
        fig1.layout.hovermode = 'x'
        # fig1.update_traces(visible = 'legendonly')

        # %% Return Graph generation
        fig2 = go.Figure()
        for i in range(len(ticker)):
            r = hist[['{}_r'.format(ticker[i])]].dropna()
            fig2.add_trace(go.Scatter(y=r['{}_r'.format(ticker[i])].to_list(), x=r.index.to_list(), name=ticker[i]))

        fig2.update_layout(
            title={
                'text': 'Historical Returns',
                'y': 0.95,
                'x': 0.5,
                'font': {'size': 22}
            },
            paper_bgcolor='white',
            plot_bgcolor='white',
            autosize=False,
            height = 450,
            xaxis={**xformat, **{
                'title': 'Closing Date',
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black',
            }},
            yaxis={
                'title': 'Return',
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black'
            },
            legend={
                'orientation': 'h',  # 图例横着排列还是竖着排列：h stands for horizontal
                # This orients the legend but it will result in an overlay of the graph
                # 'yanchor': 'top', 'y': 0.5,
                'yanchor': 'top', 'y': 1.15,  # 'top' 指上端在整个图里位置的比例（可以在0-1范围之外）
                'xanchor': 'right', 'x': 1,  # ‘right’指右端在整个图位置的比例
            },
            # this fixes the overlay
            margin={'t': 100},
        )
        fig2.layout.hovermode = 'x'
        # fig2.update_traces(visible = 'legendonly')

        return html.Div([
            dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2),
        ])
    else:
        return None


@app.callback(
    Output('strategy-list', 'style'),
    Output('strategy', 'value'),
    Input('prdata', 'data'), 
    Input('strategy', 'value'), prevent_initial_call=True)
def update_strategy(data, value):
    if data:
        time.sleep(0.5)
        return {'visibility': 'visible'}, value
    else:
        return {'visibility': 'hidden'}, None
    

    
@app.callback(
    Output('parameters', 'style'),
    Output('window-size', 'style'),
    Output('short-window-size', 'style'),
    Output('long-window-size', 'style'),
    Output('k-value', 'style'),
    Output('L-value', 'style'),
    Output('threshold', 'style'),
    Input('prdata', 'data'),
    Input('strategy', 'value'),)
def select_parameter(data, strategy):
    show = {'display': 'flex', 'flex-direction': 'row', 'padding': '1em'}
    not_show = {'display': 'none'}
    if data == None:
        return {'visibility':'hidden'}, not_show, not_show, not_show, not_show, not_show, not_show
    if strategy == 'FR':
        return {'visibility':'visible'}, show, not_show, not_show, not_show, not_show, show
    elif strategy == 'SMA':
        return {'visibility':'visible'}, not_show, show, show, not_show, not_show, show
    elif strategy == 'EMA':
        return {'visibility':'visible'}, not_show, show, show, not_show, not_show, show
    elif strategy == 'CB':
        return {'visibility':'visible'}, show, not_show, not_show, show, show, not_show
    else:
        return {'visibility':'hidden'}, not_show, not_show, not_show, not_show, not_show, not_show

@app.callback(
    Output('hist-result-plot', 'children'),
    Output('hist-result-table', 'children'),
    Output('button2', 'n_clicks'),
    Input('button2', 'n_clicks'),
    Input('prdata', 'data'),
    State('strategy', 'value'),
    State('WS', 'value'),
    State('SWS', 'value'),
    State('LWS', 'value'),
    State('K', 'value'),
    State('L', 'value'),
    State('THRES', 'value'), prevent_initial_call=True)
def testing_result(n_clicks, data, strategy, WS, SWS, LWS, K, L, THRES):
    if data == None or n_clicks == 0:
        return None, None, 0
    hist = pd.read_json(data)
    if strategy == 'FR':
        if WS and THRES != None:
            s,r = filter_rules(hist, WS, THRES)
            x = hist.index[WS : (len(hist)-1)]
        else: 
            return None, None, 0
    elif strategy == 'SMA':
        if SWS and LWS and THRES != None:
            s,r = SMA(hist, SWS, LWS, THRES)
            x = hist.index[(LWS-1) : (len(hist)-1)]
        else:
            return None, None, 0
    elif strategy == 'EMA':
        if SWS and LWS and THRES != None:
            s,r = EMA(hist, SWS, LWS, THRES)
            x = hist.index[(LWS-1) : (len(hist)-1)]
        else:
            return None, None, 0
    elif strategy == 'CB':
        if WS and K != None and L:
            s,r = channel_breakouts(hist, WS, K, L)
            x = hist.index[max(WS, L) : (len(hist)-1)]
        else:
            return None, None, 0
    else:
        return None, None, 0
    
    #$$ plot generation
    ticker = s.columns
    xformat = dict(rangeslider=dict(visible=True), type="date")
    
    fig1 = go.Figure()
    for i in range(len(ticker)):
        fig1.add_trace(go.Scatter(y=s[ticker[i]].to_list(), x=x.to_list(), name=ticker[i], mode='markers'))

    fig1.update_layout(
        title={
            'text': 'Daily Holding Position',
            'y': 0.95,
            'x': 0.5,
            'font': {'size': 22}
        },
        paper_bgcolor='white',
        plot_bgcolor='white',
        autosize=False,
        height = 450,
        xaxis={**xformat, **{
            'title': 'Closing Date',
            'showline': True,
            'linewidth': 1,
            'linecolor': 'black',
        }},
        yaxis={
            'title': 'Position',
            'showline': True,
            'linewidth': 1,
            'linecolor': 'black'
        },
        legend={
            'orientation': 'h',  # 图例横着排列还是竖着排列：h stands for horizontal
            # This orients the legend but it will result in an overlay of the graph
            # 'yanchor': 'top', 'y': 0.5,
            'yanchor': 'top', 'y': 1.15,  # 'top' 指上端在整个图里位置的比例（可以在0-1范围之外）
            'xanchor': 'right', 'x': 1,  # ‘right’指右端在整个图位置的比例
        },
        # this fixes the overlay
        margin={'t': 100},
    )
    fig1.update_traces(marker = {'symbol': 'circle-dot'})
    fig1.layout.hovermode = 'x'
    # fig1.update_traces(visible = 'legendonly')

    # %% Return Graph generation
    fig2 = go.Figure()
    for i in range(len(ticker)):
        fig2.add_trace(go.Scatter(y=r[ticker[i]].to_list(), x=x.to_list(), name=ticker[i]))

    fig2.update_layout(
        title={
            'text': 'Achieved Daily Returns',
            'y': 0.95,
            'x': 0.5,
            'font': {'size': 22}
        },
        paper_bgcolor='white',
        plot_bgcolor='white',
        autosize=False,
        height = 450,
        xaxis={**xformat, **{
            'title': 'Closing Date',
            'showline': True,
            'linewidth': 1,
            'linecolor': 'black',
        }},
        yaxis={
            'title': 'Return',
            'showline': True,
            'linewidth': 1,
            'linecolor': 'black'
        },
        legend={
            'orientation': 'h',  # 图例横着排列还是竖着排列：h stands for horizontal
            # This orients the legend but it will result in an overlay of the graph
            # 'yanchor': 'top', 'y': 0.5,
            'yanchor': 'top', 'y': 1.15,  # 'top' 指上端在整个图里位置的比例（可以在0-1范围之外）
            'xanchor': 'right', 'x': 1,  # ‘right’指右端在整个图位置的比例
        },
        # this fixes the overlay
        margin={'t': 100},
    )
    fig2.layout.hovermode = 'x'
    # fig2.update_traces(visible = 'legendonly')
        
        
    #generate result table
    history_result = pd.DataFrame(columns = ticker)
    for i in range(len(ticker)):
        history_result[ticker[i]] = metrics(s[ticker[i]], r[ticker[i]])
    metrics_list = ['Ticker', 'Round-trip trades', 'Total return (%)', 'Buy & hold return (%)', 
                    'Winning trades (%)', 'Ratio of average winning amount to average losing amount (%)', 
                    'T-value', 'Maximum drawdown (%)']
    return html.Div([dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]), html.Table([
        html.Thead(
            html.Tr([html.Th(metric) for metric in metrics_list])
        ),
        html.Tbody([
                html.Tr([html.Td(ticker[i])] + [html.Td(x) for x in history_result[ticker[i]]]) for i in range(len(ticker))
        ]),
        
    ], style={'width': '32em'}), n_clicks

    
def filter_rules(df, h_length, thre):
    num = int(len(df.columns)/2)
    ticker = df.columns[:num]
    realized_r = pd.DataFrame(columns = ticker)
    strategy = pd.DataFrame(columns = ticker)
    for n in range(num):
        stra = []
        rr = []
        for i in range(h_length, (len(df)-1)):
            if df[ticker[n]][i]/max(df[ticker[n]][(i-h_length):i]) > (1 + thre):
                stra.append(1)
                rr.append(df['{}_r'.format(ticker[n])][i+1])
            elif df[ticker[n]][i]/min(df[ticker[n]][(i-h_length):i]) < (1 - thre):
                stra.append(-1)
                rr.append(-df['{}_r'.format(ticker[n])][i+1])
            else:
                stra.append(0)
                rr.append(0)
        realized_r[ticker[n]] = rr
        strategy[ticker[n]] = stra
    return strategy, realized_r

    
    
def SMA(df, sn, ln, thre):
    num = int(len(df.columns)/2)
    ticker = df.columns[:num]
    realized_r = pd.DataFrame(columns = ticker)
    strategy = pd.DataFrame(columns = ticker)
    for n in range(num):
        stra = []
        rr = []
        for i in range((ln-1), (len(df)-1)):
            short = np.mean(df[ticker[n]][(i-sn+1):(i+1)])
            long = np.mean(df[ticker[n]][(i-ln+1):(i+1)])
            r = (short - long)/long
            if r > thre:
                stra.append(1)
                rr.append(df['{}_r'.format(ticker[n])][i+1])
            elif r < -thre:
                stra.append(-1)
                rr.append(-df['{}_r'.format(ticker[n])][i+1])
            else:
                stra.append(0)
                rr.append(0)
        realized_r[ticker[n]] = rr
        strategy[ticker[n]] = stra
    return strategy, realized_r
    
    
    
def EMA(df, sn, ln, thre):
    num = int(len(df.columns)/2)
    ticker = df.columns[:num]
    realized_r = pd.DataFrame(columns = ticker)
    strategy = pd.DataFrame(columns = ticker)
    beta_s = 2/(sn+1)
    beta_l = 2/(ln+1)
    for n in range(num):
        stra = []
        rr = []
        for i in range((ln-1), (len(df)-1)):
            short = 0
            long = 0
            multiplier_s = beta_s
            multiplier_l = beta_l
            for d in range(i, i-sn, -1):
                short += multiplier_s*df[ticker[n]][d]
                multiplier_s *= (1-beta_s)
            for d in range(i, i-ln, -1):
                long += multiplier_l*df[ticker[n]][d]
                multiplier_l *= (1-beta_l)
            r = (short - long)/long
            if r > thre:
                stra.append(1)
                rr.append(df['{}_r'.format(ticker[n])][i+1])
            elif r < -thre:
                stra.append(-1)
                rr.append(-df['{}_r'.format(ticker[n])][i+1])
            else:
                stra.append(0)
                rr.append(0)
        realized_r[ticker[n]] = rr
        strategy[ticker[n]] = stra
    return strategy, realized_r


def channel_breakouts(df, h_length, k, L):
    num = int(len(df.columns)/2)
    ticker = df.columns[:num]
    realized_r = pd.DataFrame(columns = ticker)
    strategy = pd.DataFrame(columns = ticker)
    start_i = max(h_length, L)
    for n in range(num):
        stra = []
        rr = []
        for i in range(start_i, (len(df)-1)):
            B = k*np.std(df['{}_r'.format(ticker[n])][(i-L+1):(i+1)])
            maximum = max(df[ticker[n]][(i-h_length):i])
            minimum = min(df[ticker[n]][(i-h_length):i])
            if i == start_i or (i > start_i and stra[-1] == 0):
                if df[ticker[n]][i] > (1+B)*maximum:
                    stra.append(1)
                    rr.append(df['{}_r'.format(ticker[n])][i+1])
                elif df[ticker[n]][i] < (1-B)*minimum:
                    stra.append(-1)
                    rr.append(-df['{}_r'.format(ticker[n])][i+1])
                else:
                    stra.append(0)
                    rr.append(0)
            elif stra[-1] == 1:
                if df[ticker[n]][i] < (1-B)*minimum:
                    stra.append(0)
                    rr.append(0)
                else:
                    stra.append(1)
                    rr.append(df['{}_r'.format(ticker[n])][i+1])
            else:
                if df[ticker[n]][i] > (1+B)*maximum:
                    stra.append(0)
                    rr.append(0)
                else:
                    stra.append(-1)
                    rr.append(-df['{}_r'.format(ticker[n])][i+1])
        realized_r[ticker[n]] = rr
        strategy[ticker[n]] = stra
    return strategy, realized_r



def metrics(strategy, realized_r):
    w = [i for i in realized_r if i > 0]
    l = [i for i in realized_r if i < 0]
    ratio = np.mean(w)/abs(np.mean(l))*100
    roundtrip_trade = 0
    for i in range(len(strategy)):
        if i == 0:
            if strategy[i] != 0:
                roundtrip_trade += 0.5
        else:
            if abs(strategy[i] - strategy[i-1]) == 1:
                roundtrip_trade += 0.5
            elif abs(strategy[i] - strategy[i-1]) == 2:
                roundtrip_trade += 1
    accumulated_return = np.cumsum(realized_r)+np.ones(len(realized_r))
    total_return = np.sum(realized_r)*100
    BH_return = (np.prod(realized_r+np.ones(len(realized_r)))-1)*100
    winning_trades = len(w)/len(realized_r)*100
    t_value = np.mean(realized_r)/(np.var(realized_r)/len(realized_r))**0.5
    max_drawdown = 0
    for i in range(len(realized_r)):
        maximum = max(accumulated_return[:(i+1)])
        if maximum - accumulated_return[i] > max_drawdown:
            max_drawdown = maximum - accumulated_return[i]
    return [roundtrip_trade, total_return, BH_return, winning_trades, ratio, t_value, max_drawdown*100]
    


# This is always the last line where the app is actually instanciated
app.run_server(
    mode = 'inline',
    port = 8011
)

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  4 of 4 completed


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from scipy import stats
import statsmodels.api as sm
import yfinance as yf
import dash
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
from datetime import date
import time
from dash.exceptions import PreventUpdate
ticker=['TCEHY']
start_d = '2017-01-01'
end_d = '2020-01-01'
myTicker = ' '
myTicker = myTicker.join(ticker)
hist = yf.download(myTicker, start=start_d, end=end_d)
hist

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,24.350000,24.540001,24.250000,24.450001,24.143795,802200
2017-01-04,24.480000,24.580000,24.350000,24.549999,24.242542,977800
2017-01-05,25.080000,25.180000,24.950001,25.080000,24.765903,805500
2017-01-06,25.330000,25.350000,25.059999,25.200001,24.884401,745600
2017-01-09,25.299999,25.299999,25.120001,25.280001,24.963400,455300
...,...,...,...,...,...,...
2019-12-24,48.340000,48.450001,48.310001,48.340000,48.081791,572600
2019-12-26,48.490002,48.869999,48.419998,48.810001,48.549282,1584300
2019-12-27,49.099998,49.360001,49.080002,49.240002,48.976990,2065900


In [3]:
hist['Adj Close'][['AAPL']] #双重括号保证dataframe而不是series


,AAPL
Date,
2017-01-03,27.332470
2017-01-04,27.301882
2017-01-05,27.440718
2017-01-06,27.746632
2017-01-09,28.000782
...,...
2019-12-24,70.027306
2019-12-26,71.416679
2019-12-27,71.389572


In [1]:
[1]*7

[1, 1, 1, 1, 1, 1, 1]